## Usage
Used to get intermediate from SBBGCA(including score, member, apis...)

## preRequirement
Have to run SBBGCAExample - startClustering() function and get two pickles first

In [1]:
tag = "allaple_woj_g_98"  # used for pickle name
outputPath = "output/SBBGCA_Clustering/allaple_woj_g_98/"

In [2]:
import pickle

# read the results from pickle files
with open(outputPath + tag + '_intermediate.pickle', 'rb') as handle:
    intermediate = pickle.load(handle)
with open(outputPath + tag + '_initialDict.pickle', 'rb') as handle:
    initialDict = pickle.load(handle)
with open(outputPath + tag + '_roundInfos.pickle', 'rb') as handle:
    roundInfos = pickle.load(handle)

In [3]:
# calculate motif lengths of all common motifs
def getMotifsLengthList(motifs):
    motifLens = list()
    for motif in motifs:
        startIdx =motif[1]
        endIdx = motif[2]
        mLen = endIdx - startIdx + 1
        motifLens.append(mLen)
    return motifLens

In [4]:
def findGeneratedRoundNumber(clusterName, roundInfosDict):
    for key, value in roundInfosDict.items():
        if clusterName in value:
            return key
    return -1

In [5]:
import csv

descendant_dict = dict()
groupInfo_list = list()
groupMotif_dict = dict()

intermediate_list = sorted(intermediate.items(), key=lambda x : x[0])
for item in intermediate_list:
    value = item[1] # get original dict value
    score = value[0]
    clusterName = value[1][0]
    memberSet = value[2]
    motifs = value[1][1]
    
    # calculate motif lengths of all common motifs
    motifLens = getMotifsLengthList(motifs)

    motifsCount = len(motifs)
    # merge descendants
    descendants = set() # a new set to save descendants
    for member in memberSet:
        if member[0] == "G": # if a cluster member is already a grouped cluster
            for descendant in descendant_dict[member] :
                descendants.add(descendant)
        else:
            descendants.add(member)

    descendant_dict[clusterName] = descendants
    descendantSize = len(descendant_dict)
    
    groupMotif_dict[clusterName] = motifs
    
    roundNumber = findGeneratedRoundNumber(clusterName, roundInfos)
    
    groupInfo_list.append((roundNumber, clusterName, score, memberSet, motifsCount, motifLens))
    
# write file "GroupInfo.csv" :  clusterName, score, members, motifCount, common motifs length list
with open(outputPath + tag + "_GroupInfo.csv", 'w', newline='') as infoFile:
    spamwriter = csv.writer(infoFile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    header = ["Round", "ClusterName", "SimilarityScore", "Members", "MotifsCount", "MotifLength"]
    spamwriter.writerow(header)
    
    # write initial cluster informations(i.e., hooklogs)
    for key in sorted(initialDict.keys(), key = lambda x : int(x[1::])):
        # something like this: (0, "G1", "N/A", "abc", 1, 109)
        originDataRow = (0, key, "N/A", initialDict[key][0], 1, initialDict[key][1])
        spamwriter.writerow(originDataRow)
        
    # write cluster informations
    for group in groupInfo_list:
        spamwriter.writerow(group)
    
    
# write file "Descendants.csv" :  clusterName, descendants size,  descendants list
with open(outputPath + tag + "_Descendants.csv", 'w', newline='') as desFile:
    spamwriter = csv.writer(desFile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    header = ["ClusterName", "DescendantSize", "Descendants"]
    spamwriter.writerow(header)
    for key in sorted(descendant_dict.keys(), key = lambda x : int(x[1::])):
        row = (key, len(descendant_dict[key]), descendant_dict[key])
        spamwriter.writerow(row)
        
# write file "Motifs.csv" :  clusterName, MotifNumber, apis
with open(outputPath + tag + "_Motifs.csv", 'w', newline='') as motifFile:
    spamwriter = csv.writer(motifFile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    header = ["ClusterName", "MotifNumber", "APIs"]
    spamwriter.writerow(header)

    for key in sorted(groupMotif_dict.keys(), key = lambda x : int(x[1::])):
        group_motifs = groupMotif_dict[key]
        motifNumber = 0
        for motif in group_motifs:
            firstMotifAPI = True
            for api in motif[0]:
                if(firstMotifAPI):
                    row = (key, motifNumber, api)
                    firstMotifAPI = False
                else:
                    row = ("", "", api)
                spamwriter.writerow(row)
            motifNumber += 1
            
# # write file "RoundInformation.csv" : Round, Generate Group
# with open(outputPath + tag + "_RoundInformation.csv", 'w', newline='') as roundInfoFile:
#     spamwriter = csv.writer(roundInfoFile, delimiter=',',
#                             quotechar='"', quoting=csv.QUOTE_MINIMAL)
#     header = ["Round", "Generate Group"]
#     spamwriter.writerow(header)
    
#     for key in sorted(roundInfos.keys(), key = lambda x:int(x)):
#         row = (key, roundInfos[key])
#         spamwriter.writerow(row)

In [6]:
# output residual information of SBBGCA

with open(outputPath + tag + '_residual.pickle', 'rb') as handle:
    residual = pickle.load(handle)
    
with open(outputPath + tag + "_GroupInfo.csv", 'a', newline='') as expandGroupInfo:
    spamwriter = csv.writer(expandGroupInfo, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    header = ["ClusterName", "Members", "MotifLength"]
    
    spamwriter.writerow("")
    spamwriter.writerow(("Residual Clusters:","",""))
    spamwriter.writerow(header)
    
    for key, value in residual.items():
        clusterName = value[0][0]
        motifsList = value[0][1]
        motifLens = getMotifsLengthList(motifsList)
        members = value[1]
        if( len(members) == 0 ):
            row = (clusterName, "N/A", motifLens)
        else:
            row = (clusterName, members, motifLens)
            
        spamwriter.writerow(row)